In [7]:
# -*- coding: utf-8 -*-
"""packt-18-node-classification.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/11ZGUyaZojomSjmSN58eWmBZjQ4GPjvMu

# Node Classification with DGL

Tensorflow and DGL re-implementation of the PyTorch and DGL based example at [Node Classification with DGL](https://docs.dgl.ai/tutorials/blitz/1_introduction.html#sphx-glr-tutorials-blitz-1-introduction-py)
"""

# Commented out IPython magic to ensure Python compatibility.
# %env DGLBACKEND=tensorflow

!pip install dgl tensorflow-addons

# Commented out IPython magic to ensure Python compatibility.
import dgl
import dgl.data
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
import tensorflow_addons as tfa

from dgl.nn.tensorflow import GraphConv

# %matplotlib inline

"""## Loading CORA dataset"""

dataset = dgl.data.CoraGraphDataset()
print('Number of categories:', dataset.num_classes)

# cora dataset contains one graph in dataset
len(dataset)

g = dataset[0]

print('Node features')
print(g.ndata)
print('Edge features')
print(g.edata)

"""## Defining a Graph Convolutional Network (GCN)"""

class NodeClassifier(tf.keras.Model):
  def __init__(self, g, in_feats, h_feats, num_classes):
    super(NodeClassifier, self).__init__()
    self.g = g
    self.conv1 = GraphConv(in_feats, h_feats, activation=tf.nn.relu)
    self.conv2 = GraphConv(h_feats, num_classes)

  def call(self, in_feat):
    h = self.conv1(self.g, in_feat)
    h = self.conv2(self.g, h)
    return h


g = dataset[0]
model = NodeClassifier(g, g.ndata["feat"].shape[1], 16, dataset.num_classes)

"""## Training the GCN"""

device = "/cpu:0"
gpus = tf.config.list_physical_devices("GPU")
if len(gpus) > 0:
  device = gpus[0]
g = g.to(device)

def do_eval(model, features, labels, mask):
  logits = model(features, training=False)
  logits = logits[mask]
  labels = labels[mask]
  preds = tf.math.argmax(logits, axis=1)
  acc = tf.reduce_mean(tf.cast(preds == labels, dtype=tf.float32))
  return acc.numpy().item()

NUM_HIDDEN = 16
LEARNING_RATE = 1e-2
WEIGHT_DECAY = 5e-4
NUM_EPOCHS = 200

with tf.device(device):
  feats = g.ndata["feat"]
  labels = g.ndata["label"]
  train_mask = g.ndata["train_mask"]
  val_mask = g.ndata["val_mask"]
  test_mask = g.ndata["test_mask"]
  in_feats = feats.shape[1]
  n_classes = dataset.num_classes
  n_edges = dataset[0].number_of_edges()

  model = NodeClassifier(g, in_feats, NUM_HIDDEN, n_classes)
  loss_fcn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
  optimizer = tfa.optimizers.AdamW(
      learning_rate=LEARNING_RATE, weight_decay=WEIGHT_DECAY)

  best_val_acc, best_test_acc = 0, 0
  history = []
  for epoch in range(NUM_EPOCHS):
    with tf.GradientTape() as tape:
      logits = model(feats)
      loss = loss_fcn(labels[train_mask], logits[train_mask])
      grads = tape.gradient(loss, model.trainable_weights)
      optimizer.apply_gradients(zip(grads, model.trainable_weights))
  
    val_acc = do_eval(model, feats, labels, val_mask)
    history.append((epoch + 1, loss.numpy().item(), val_acc))

    if epoch % 10 == 0:
      print("Epoch {:3d} | train loss: {:.3f} | val acc: {:.3f}".format(
          epoch, loss.numpy().item(), val_acc))

epochs = [epoch for epoch, _, _ in history]
losses = [loss for _, loss, _ in history]
val_accs = [val_acc for _, _, val_acc in history]

plt.subplot(2, 1, 1)
plt.plot(epochs, losses)
plt.xlabel("epochs")
plt.ylabel("train loss")

plt.subplot(2, 1, 2)
plt.plot(epochs, val_accs)
plt.xlabel("epochs")
plt.ylabel("val acc")

plt.tight_layout()
_ = plt.show()

test_acc = do_eval(model, feats, labels, test_mask)
print("Test acc: {:.3f}".format(test_acc))



Defaulting to user installation because normal site-packages is not writeable


ModuleNotFoundError: No module named 'dgl.nn.tensorflow'

In [ ]:
!git clone --recurse-submodules https://github.com/dmlc/dgl.git